#Azure Storage Service Demo
If you have not installed azure and azure-storage un-comment the pip below.

In [1]:
#!pip install --upgrade azure-storage

In [1]:
import csv
import sys
import azure.storage
from azure.storage.table import TableService, Entity
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess
import time
print "hello at" + time.asctime(time.localtime(time.time()))

hello atThu Jul 07 16:28:32 2016


follow the instructions in the book to create a storage account.   the one used here is "escistore", but you will want to pick something else and replace that name everywhere below

In [9]:
block_blob_service = BlockBlobService(account_name='escistore',
    account_key='Your account key')
block_blob_service.create_container('datacont', 
                                    public_access=PublicAccess.Container)

True

In [3]:
table_service = TableService(account_name='escistore', 
                             account_key='your storage account key... same as above')

In [4]:
if table_service.create_table('DataTable'):
    print "table created"
else:
    print "table already there"

table already there


Now you need a directory with a csv file.  For example

'experiment1', '1', '3/15/2002', 'exp1', 'this is the comment'

'experiment1', '2', '3/15/2002', 'exp2', 'this is the comment2'

'experiment2', '3', '3/16/2002', 'exp3', 'this is the comment3'

'experiment3', '4', '3/16/2002', 'exp4', 'this is the comment233'

now you need a directory datafiles that has four blobs of any type.  call them exp1, exp2, exp3, exp4.  
the code below will upload the data blobs into the container "datacont", then create the url and upload the metadata into the table "DataTable".

In [ ]:
 with open('c:\users\dennis\documents\experiments.csv', 'rb') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        print item
        block_blob_service.create_blob_from_path(
            'datacont', item[3],
            "c:\users\dennis\documents\datafiles\\"+item[3]
            )
        url = "https://escistore.blob.core.windows.net/datacont/"+item[3]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1], 
                 'description' : item[4], 'date' : item[2], 'url':url} 
        table_service.insert_entity('DataTable', metadata_item)
        

Next let's do a querry for experiment1 and project onto the urls.  it should print the urls created above

In [ ]:
tasks = table_service.query_entities('DataTable', filter="PartitionKey eq 'experiment1'", select='url')
for task in tasks:
    print(task.url)

The output should look like

https://escistore.blob.core.windows.net/datacont/exp1

https://escistore.blob.core.windows.net/datacont/exp2
